# Evaluating AI with Haystack

by Bilge Yucel ([X](https://x.com/bilgeycl), [Linkedin](https://www.linkedin.com/in/bilge-yucel/))

In this cookbook, we walk through the [Evaluators](https://docs.haystack.deepset.ai/docs/evaluators) in Haystack, create an evaluation pipeline, streamline the evaluation with [`EvaluationHarness`](https://github.com/deepset-ai/haystack-experimental/tree/main/haystack_experimental/evaluation/harness) and try different Evaluation Frameworks like [Ragas](https://haystack.deepset.ai/integrations/ragas) and [FlowJudge](https://haystack.deepset.ai/integrations/flow-judge). 

📚 **Useful Resources:**
* [Article: Benchmarking Haystack Pipelines for Optimal Performance](https://haystack.deepset.ai/blog/benchmarking-haystack-pipelines)
* [Evaluation Walkthrough](https://haystack.deepset.ai/tutorials/guide_evaluation)
* [haystack-evaluation repo](https://github.com/deepset-ai/haystack-evaluation/tree/main)
* [Evaluation tutorial](https://haystack.deepset.ai/tutorials/35_evaluating_rag_pipelines)
* [Evaluation Docs](https://docs.haystack.deepset.ai/docs/evaluation)

## 📺 Watch Along

<iframe width="560" height="315" src="https://www.youtube.com/embed/Dy-n_yC3Cto?si=LB0GdFP0VO-nJT-n" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

In [ ]:
!pip install haystack-ai "sentence-transformers>=3.0.0" pypdf

In [ ]:
!pip install ragas-haystack "flow-judge[hf]" # evaluation frameworks

## 1. Building your pipeline

### ARAGOG

This dataset is based on the paper [Advanced Retrieval Augmented Generation Output Grading (ARAGOG)](https://arxiv.org/pdf/2404.01037). It's a
collection of papers from ArXiv covering topics around Transformers and Large Language Models, all in PDF format.

The dataset contains:
- 13 PDF papers.
- 107 questions and answers generated with the assistance of GPT-4, and validated/corrected by humans.

We have:
- ground-truth answers
- questions

Get the dataset [here](https://github.com/deepset-ai/haystack-evaluation/blob/main/datasets/README.md)

In [9]:
# Run this to download the dataset
!mkdir -p ARAGOG/papers_for_questions

!wget https://raw.githubusercontent.com/deepset-ai/haystack-evaluation/main/datasets/ARAGOG/papers_for_questions/DetectGPT.pdf -P ARAGOG/papers_for_questions
!wget https://raw.githubusercontent.com/deepset-ai/haystack-evaluation/main/datasets/ARAGOG/papers_for_questions/MMLU_measure.pdf -P ARAGOG/papers_for_questions
!wget https://raw.githubusercontent.com/deepset-ai/haystack-evaluation/main/datasets/ARAGOG/papers_for_questions/PAL.pdf -P ARAGOG/papers_for_questions
!wget https://raw.githubusercontent.com/deepset-ai/haystack-evaluation/main/datasets/ARAGOG/papers_for_questions/bert.pdf -P ARAGOG/papers_for_questions
!wget https://raw.githubusercontent.com/deepset-ai/haystack-evaluation/main/datasets/ARAGOG/papers_for_questions/codenet.pdf -P ARAGOG/papers_for_questions
!wget https://raw.githubusercontent.com/deepset-ai/haystack-evaluation/main/datasets/ARAGOG/papers_for_questions/distilbert.pdf -P ARAGOG/papers_for_questions
!wget https://raw.githubusercontent.com/deepset-ai/haystack-evaluation/main/datasets/ARAGOG/papers_for_questions/glm_130b.pdf -P ARAGOG/papers_for_questions
!wget https://raw.githubusercontent.com/deepset-ai/haystack-evaluation/main/datasets/ARAGOG/papers_for_questions/hellaswag.pdf -P ARAGOG/papers_for_questions
!wget https://raw.githubusercontent.com/deepset-ai/haystack-evaluation/main/datasets/ARAGOG/papers_for_questions/llama.pdf -P ARAGOG/papers_for_questions
!wget https://raw.githubusercontent.com/deepset-ai/haystack-evaluation/main/datasets/ARAGOG/papers_for_questions/llm_long_tail.pdf -P ARAGOG/papers_for_questions
!wget https://raw.githubusercontent.com/deepset-ai/haystack-evaluation/main/datasets/ARAGOG/papers_for_questions/meaning_of_prompt.pdf -P ARAGOG/papers_for_questions
!wget https://raw.githubusercontent.com/deepset-ai/haystack-evaluation/main/datasets/ARAGOG/papers_for_questions/megatron.pdf -P ARAGOG/papers_for_questions
!wget https://raw.githubusercontent.com/deepset-ai/haystack-evaluation/main/datasets/ARAGOG/papers_for_questions/red_teaming.pdf -P ARAGOG/papers_for_questions
!wget https://raw.githubusercontent.com/deepset-ai/haystack-evaluation/main/datasets/ARAGOG/papers_for_questions/roberta.pdf -P ARAGOG/papers_for_questions
!wget https://raw.githubusercontent.com/deepset-ai/haystack-evaluation/main/datasets/ARAGOG/papers_for_questions/superglue.pdf -P ARAGOG/papers_for_questions
!wget https://raw.githubusercontent.com/deepset-ai/haystack-evaluation/main/datasets/ARAGOG/papers_for_questions/task2vec.pdf -P ARAGOG/papers_for_questions


--2025-07-08 14:57:31--  https://raw.githubusercontent.com/deepset-ai/haystack-evaluation/main/datasets/ARAGOG/papers_for_questions/DetectGPT.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2104936 (2,0M) [application/octet-stream]
Saving to: ‘ARAGOG/papers_for_questions/DetectGPT.pdf’

DetectGPT.pdf       100%[===================>]   2,01M  --.-KB/s    in 0,1s    

2025-07-08 14:57:31 (19,0 MB/s) - ‘ARAGOG/papers_for_questions/DetectGPT.pdf’ saved [2104936/2104936]

--2025-07-08 14:57:31--  https://raw.githubusercontent.com/deepset-ai/haystack-evaluation/main/datasets/ARAGOG/papers_for_questions/MMLU_measure.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.git

### Indexing Pipeline

In [10]:
import os

from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.converters import PyPDFToDocument
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy

embedding_model="sentence-transformers/all-MiniLM-L6-v2"
document_store = InMemoryDocumentStore()

files_path = "./ARAGOG/papers_for_questions" # <ENTER YOUR PATH HERE>
pipeline = Pipeline()
pipeline.add_component("converter", PyPDFToDocument())
pipeline.add_component("cleaner", DocumentCleaner())
pipeline.add_component("splitter", DocumentSplitter(split_length=250, split_by="word"))  # default splitting by word
pipeline.add_component("writer", DocumentWriter(document_store=document_store, policy=DuplicatePolicy.SKIP))
pipeline.add_component("embedder", SentenceTransformersDocumentEmbedder(embedding_model))
pipeline.connect("converter", "cleaner")
pipeline.connect("cleaner", "splitter")
pipeline.connect("splitter", "embedder")
pipeline.connect("embedder", "writer")
pdf_files = [files_path + "/" + f_name for f_name in os.listdir(files_path)]

pipeline.run({"converter": {"sources": pdf_files}})

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

{'writer': {'documents_written': 691}}

In [11]:
document_store.count_documents()

691

### RAG

In [12]:
import os
from getpass import getpass

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass('OPENAI_API_KEY: ')

In [13]:
from haystack import Pipeline
from haystack.components.builders import ChatPromptBuilder, AnswerBuilder
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.retrievers import InMemoryEmbeddingRetriever
from haystack.dataclasses import ChatMessage

chat_message = ChatMessage.from_user(
    text="""You have to answer the following question based on the given context information only.
If the context is empty or just a '\\n' answer with None, example: "None".

Context:
{% for document in documents %}
  {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""
)

basic_rag = Pipeline()
basic_rag.add_component("query_embedder", SentenceTransformersTextEmbedder(
    model=embedding_model, progress_bar=False
))
basic_rag.add_component("retriever", InMemoryEmbeddingRetriever(document_store))
basic_rag.add_component("chat_prompt_builder", ChatPromptBuilder(template=[chat_message], required_variables="*"))
basic_rag.add_component("chat_generator", OpenAIChatGenerator(model="gpt-4o-mini"))

basic_rag.connect("query_embedder", "retriever.query_embedding")
basic_rag.connect("retriever", "chat_prompt_builder.documents")
basic_rag.connect("chat_prompt_builder", "chat_generator")

🚅 Components
  - query_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - chat_prompt_builder: ChatPromptBuilder
  - chat_generator: OpenAIChatGenerator
🛤️ Connections
  - query_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> chat_prompt_builder.documents (List[Document])
  - chat_prompt_builder.prompt -> chat_generator.messages (List[ChatMessage])

## 2. Human Evaluation

In [14]:
!wget https://raw.githubusercontent.com/deepset-ai/haystack-evaluation/main/datasets/ARAGOG/eval_questions.json -P ARAGOG


--2025-07-08 14:58:29--  https://raw.githubusercontent.com/deepset-ai/haystack-evaluation/main/datasets/ARAGOG/eval_questions.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


200 OK
Length: 37510 (37K) [text/plain]
Saving to: ‘ARAGOG/eval_questions.json’

eval_questions.json 100%[===================>]  36,63K  --.-KB/s    in 0,02s   

2025-07-08 14:58:29 (2,13 MB/s) - ‘ARAGOG/eval_questions.json’ saved [37510/37510]



In [15]:
from typing import List, Tuple
import json

def read_question_answers() -> Tuple[List[str], List[str]]:
    with open("./ARAGOG/eval_questions.json", "r") as f:
        data = json.load(f)
        questions = data["questions"]
        answers = data["ground_truths"]
    return questions, answers

all_questions, all_answers = read_question_answers()

In [16]:
print(len(all_questions))
print(len(all_answers))

107
107


In [17]:
questions = all_questions[:15]
answers = all_answers[:15]

In [18]:
index = 5
print(questions[index])
print(answers[index])
question = questions[index]

How were the questions for the multitask test sourced, and what was the criteria for their inclusion?
Questions were manually collected by graduate and undergraduate students from freely available online sources, including practice questions for standardized tests and undergraduate courses, ensuring a wide representation of difficulty levels and subjects.


In [19]:
basic_rag.run({"query_embedder":{"text":question}, "chat_prompt_builder":{"question": question}})

{'chat_generator': {'replies': [ChatMessage(_role=<ChatRole.ASSISTANT: 'assistant'>, _content=[TextContent(text='The questions for the multitask test were manually collected by graduate and undergraduate students from freely available sources online. These sources included practice questions for tests such as the Graduate Record Examination and the United States Medical Licensing Examination, as well as questions designed for undergraduate courses and readers of Oxford University Press books. The criteria for inclusion involved ensuring that each subject contained a sufficient number of test examples, with each subject having a minimum of 100 test examples. Tasks that were either too challenging for humans without extensive training or too easy for the machine baselines were filtered out.')], _name=None, _meta={'model': 'gpt-4o-mini-2024-07-18', 'index': 0, 'finish_reason': 'stop', 'usage': {'completion_tokens': 110, 'prompt_tokens': 4550, 'total_tokens': 4660, 'completion_tokens_detai

## 3. Deciding on Metrics

* **Semantic Answer Similarity**: SASEvaluator compares the embedding of a generated answer against a ground-truth answer based on a common embedding model.
* **ContextRelevanceEvaluator** will assess the relevancy of the retrieved context to answer the query question
* **FaithfulnessEvaluator** evaluates whether the generated answer can be derived from the context


## 4. Building an Evaluation Pipeline

In [20]:
from haystack import Pipeline
from haystack.components.evaluators import ContextRelevanceEvaluator, FaithfulnessEvaluator, SASEvaluator

eval_pipeline = Pipeline()
eval_pipeline.add_component("context_relevance", ContextRelevanceEvaluator(raise_on_failure=False))
eval_pipeline.add_component("faithfulness", FaithfulnessEvaluator(raise_on_failure=False))
eval_pipeline.add_component("sas", SASEvaluator(model=embedding_model))

2025-07-08T12:58:37.632950Z [warning  ] PromptBuilder has 2 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`. length=2 lineno=186 module=haystack.components.builders.prompt_builder
2025-07-08T12:58:37.662420Z [warning  ] PromptBuilder has 3 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`. length=3 lineno=186 module=haystack.components.builders.prompt_builder


## 5. Running Evaluation

### Run the RAG Pipeline

In [21]:
predicted_answers = []
retrieved_context = []

for question in questions: # loops over 15 questions
    result = basic_rag.run(
        {"query_embedder":{"text":question}, "chat_prompt_builder":{"question": question}}, include_outputs_from={"retriever"}
    )
    predicted_answers.append(result["chat_generator"]["replies"][0].text)
    retrieved_context.append(result["retriever"]["documents"])

### Run the Evaluation

In [22]:
eval_pipeline_results = eval_pipeline.run(
    {
        "context_relevance": {"questions": questions, "contexts": retrieved_context},
        "faithfulness": {"questions": questions, "contexts": retrieved_context, "predicted_answers": predicted_answers},
        "sas": {"predicted_answers": predicted_answers, "ground_truth_answers": answers},
    }
)

results = {
    "context_relevance": eval_pipeline_results['context_relevance'],
    "faithfulness": eval_pipeline_results['faithfulness'],
    "sas": eval_pipeline_results['sas']
}

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:33<00:00,  2.23s/it]


## 6. Analyzing Results

[EvaluationRunResult](https://docs.haystack.deepset.ai/reference/evaluation-api#evaluationrunresult)

In [23]:
from haystack.evaluation import EvaluationRunResult

inputs = {
    'questions': questions,
    'contexts': retrieved_context,
    'true_answers': answers,
    'predicted_answers': predicted_answers
}
run_name="rag_eval"
eval_results = EvaluationRunResult(run_name=run_name, inputs=inputs, results=results)
eval_results.aggregated_report()

{'metrics': ['context_relevance', 'faithfulness', 'sas'],
 'score': [0.26666666666666666, 0.7, 0.5344941093275944]}

In [24]:
index = 2
print(eval_pipeline_results['context_relevance']["individual_scores"][index], "\nQuestion:", questions[index],"\nTrue Answer:", answers[index], "\nAnswer:", predicted_answers[index])
print("".join([doc.content for doc in retrieved_context[index]]))

0 
Question: How does BERT's architecture facilitate the use of a unified model across diverse NLP tasks? 
True Answer: BERT uses a multi-layer bidirectional Transformer encoder architecture, allowing for minimal task-specific architecture modifications in fine-tuning. 
Answer: BERT's architecture facilitates the use of a unified model across diverse NLP tasks through its multi-layer bidirectional Transformer encoder design. This design allows the model to pre-train deep bidirectional representations that jointly condition on both left and right context in all layers. As a result, the architecture remains largely unchanged between pre-training and fine-tuning stages, enabling minimal adjustment when adapting BERT to various downstream tasks. By incorporating only one additional output layer during fine-tuning, BERT can effectively tackle a wide range of tasks such as question answering, language inference, and named entity recognition without the need for substantial task-specific arch

## Evaluation Frameworks

* [RagasEvaluator](https://docs.haystack.deepset.ai/docs/ragasevaluator)
* [FlowJudge](https://haystack.deepset.ai/integrations/flow-judge)

In [42]:
from flow_judge.integrations.haystack import HaystackFlowJudge
from flow_judge.metrics.presets import RESPONSE_FAITHFULNESS_5POINT
from flow_judge import Hf

model = Hf(flash_attn=False)

flow_judge_evaluator = HaystackFlowJudge(
    metric=RESPONSE_FAITHFULNESS_5POINT,
    model=model,
    progress_bar=True,
    raise_on_failure=True,
    save_results=True,
    fail_on_parse_error=False
)

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2025-07-08T13:11:54.424201Z [warning  ] Some parameters are on the meta device because they were offloaded to the disk. lineno=442 module=accelerate.big_modeling
2025-07-08T13:11:54.939155Z [warning  ] Running Hf on CPU may result in longer inference times. lineno=129 module=flow_judge.models.huggingface


In [26]:
import os
from getpass import getpass

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass('OPENAI_API_KEY: ')

In [27]:
from haystack.components.generators import OpenAIGenerator
from haystack_integrations.components.evaluators.ragas import RagasEvaluator

from ragas.llms import HaystackLLMWrapper
from ragas.metrics import AnswerRelevancy, ContextPrecision, Faithfulness

llm = OpenAIGenerator(model="gpt-4o-mini")
evaluator_llm = HaystackLLMWrapper(llm)
ragas_evaluator = RagasEvaluator(
    ragas_metrics=[Faithfulness()],
    evaluator_llm=evaluator_llm,
)

In [28]:
str_fj_retrieved_context = []
for context in retrieved_context:
    str_context = [doc.content for doc in context]
    str_fj_retrieved_context.append(" ".join(str_context)) # ["", "", ...]

In [29]:
str_retrieved_context = []
for context in retrieved_context:
    str_context = [doc.content for doc in context]
    str_retrieved_context.append(str_context) # [["", ""]]

In [36]:
flow_judge_evaluator

flow_judge_evaluator
Inputs:
  - query: list[str]
  - context: list[str]
  - response: list[str]
Outputs:
  - results: list[dict[str, Any]]
  - metadata: dict[str, Any]
  - score: float
  - individual_scores: list[float]

In [ ]:
from haystack import Pipeline

integration_eval_pipeline = Pipeline()
# integration_eval_pipeline.add_component("ragas_evaluator", ragas_evaluator)
integration_eval_pipeline.add_component("flow_judge_evaluator", flow_judge_evaluator)

eval_framework_pipeline_results = integration_eval_pipeline.run(
    {
        # "ragas_evaluator": {"questions": questions, "contexts": str_retrieved_context, "responses": predicted_answers},
        "flow_judge_evaluator": {"query": questions, "context": str_fj_retrieved_context, "response": predicted_answers},
    }
)

Processing batches:   0%|                                                                                                                                       | 0/15 [00:00<?, ?it/s]/opt/homebrew/Caskroom/miniforge/base/envs/haystack_fresh/lib/python3.11/site-packages/transformers/generation/utils.py:2497: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on mps. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('mps') before running `.generate()`.
  warnings.warn(


In [ ]:
eval_framework_pipeline_results